
# Stochastic Simulation: Random Variable Generator (RVG)

In [1]:
import math
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as st
import random as rd
from operator import add

## 1. Inverse Transform Method / Direct Method / Crude Method

## 2. Rejection Method / Acceptance-Rejection Method

In [2]:
def sim_reject(fq, qs, ps, c, num:int=10):
    """Simulate realisations of discrete random variables using 
        rejection method.
    
    Keyworkd Argument
    =================
    num: Number of simulated variables.
    fq: Function to simulate realisations of discrete random 
        variables with `qs` PMF. The default value is `rd.random()`.
    qs: Probability mass function of basis random variable.
    ps: Probability mass function of desired random variable.
    """
    if not callable(fq) or not callable(qs) or not callable(ps):
        raise Exception("`fq` and `qs` must be functions!")
    
    xs = [0 for i in range(num)]
    for i in range(num):
        if_continue = True
        ## The process will continue until `xs[i]` is determined.
        while if_continue:
            y = fq()
            u = rd.random()  
            if u < ps(y) / c / qs(y):
                xs[i] = y
                if_continue = False

In [4]:
def exam_sim_reject():
    fq = rd.randint(0, 10)

In [5]:
def pmf_q_uniform(j:int):
    if j < 0 or type(j) is not int:
        raise Exception("j must be a non-negative integer!")
    elif j <= 9:
        pm = 0.1
    else:
        pm = 0
    return pm

## 2. Alias Method

In [6]:
def check_pmf(li:list):
    """Check if input probability mass function represented by a list is
    a `n`-point positive probability mass function."""
    if sum([i <= 0 for i in li]) > 0:
        raise Exception("Some probability is not positive.")
    elif abs(sum(li) - 1) >= 0.001:
        raise Exception(f"Sum of PMF {sum(li):.4f} is not 100%.")
    else:
        print(f"The input list is a {len(li)}-point positive PMF.")

In [7]:
li = [0.39, 0.2, 0.2, 0.2, 0.01]
check_pmf(li)

The input list is a 5-point positive PMF.


In [8]:
def find_ij(p):
    """Find the index i, j in the apas method for generating discrete random variables"""
    n = len(p)
    
    ## Find all indices with their values smaller than 1/(n-1) and return the first one.
    ## Note that p is considered as positive PMF, so point with 0 probablity is not considered.
    i = [i for i, x in enumerate(p) if x < 1 / (n - 1) and x > 0][0]
    
    indices = [j for j, x in enumerate(p) if p[i] + x >= 1 / (n - 1)]
    k = 0
    ## To make sure that j != i
    while indices[k] == i:
        k += 1
    j = indices[k]
    
    ## It is easy to check the results
    if j == i or p[i] >= 1 / (n - 1) or p[i] + p[j] < 1 / (n - 1):
        raise Exception("There is something wrong with the function.")
    
    ## The index in Python starts with 0
    i += 1
    j += 1
    
    return i, j

In [9]:
def set_alias(p):
    """Set up alias for `n`-point PMF."""
    n = len(p)
    ps = {n: [l for l in p]}
    qs = {}
    for k in range(1, n-1):
        i, j = find_ij(ps[n-k+1])
        i -= 1
        j -= 1
        
        qs[k] = [0 for l in range(n)]
        qs[k][i] = (n-k) * ps[n-k+1][i]
        qs[k][j] = 1 - (n-k) * ps[n-k+1][i]
        
        ps[n-k] = [(n-k) / (n-k-1) * l for l in ps[n-k+1]]
        ps[n-k][j] = (n-k) / (n-k-1) * (ps[n-k+1][i] + ps[n-k+1][j] - 1/(n-k))
        ps[n-k][i] = 0
        
        ## `ps[n-k]` should be a {n-k}-point PMF
        if abs( sum(ps[n-k]) - 1 ) > 0.0001:
            raise Exception(f"There is something wrong with {n-k}-point PMF.")
        
        ## Check the iteration
        q_t = [l / (n-k) for l in qs[k]]
        p_t = [(n-k-1) / (n-k) * l for l in ps[n-k]]
        p_tt = map(add, q_t, p_t)
        t = sum( map(add, ps[n-k+1], [-l for l in p_tt]) )
        if abs(t) > 0.001:
            print(f"t = {t}.")
            raise Exception("The function is wrong.")
        
    ## 
    k = n - 1
    qs[n - 1] = ps[2]

    ##
    q_pd = pd.DataFrame.from_dict(qs, orient='index')
    
    ## Test
    p_new = [i / (n-1) for i in q_pd.sum().tolist()]
    delta = map(add, p, [-l for l in p_new])
    t = sum([abs(l) for l in delta])
    if abs(t) > 0.001:
        print(f"t = {t}.")
        raise Exception("The final result is wrong.")
    
    return q_pd

In [10]:
def exam_set_alias():
#     p = [7/16, 0.5, 1/16]
    p = [7/16, 1/4, 1/8, 3/16]
    q_pd = set_alias(p)
    print(q_pd)
    
    
exam_set_alias()

        0     1      2       3
1  0.2500  0.75  0.000  0.0000
2  0.6250  0.00  0.375  0.0000
3  0.4375  0.00  0.000  0.5625
